In [1]:
import os
import numpy as np
from PIL import Image
import random

def read_label_file(label_file):
    with open(label_file, 'r') as f:
        lines = f.readlines()
    boxes = []
    for line in lines:
        box = list(map(float, line.strip().split()))
        boxes.append(box)
    return np.array(boxes)

def mixup(image_1_path, boxes_1, image_2_path, boxes_2, output_folder, idx):
    # 讀取圖像
    image_1 = Image.open(image_1_path).convert('RGB')
    image_2 = Image.open(image_2_path).convert('RGB')
    
    # 混合圖像
    alpha = 0.5
    mixup_image = Image.blend(image_1, image_2, alpha)
    
    # 取得檔名（不包含副檔名）
    image_1_name = os.path.splitext(os.path.basename(image_1_path))[0]
    image_2_name = os.path.splitext(os.path.basename(image_2_path))[0]
    
    # 保存混合後的圖像
    mixup_image_path = os.path.join(output_folder, f'mixup_{idx}.png')
    mixup_image.save(mixup_image_path)
    
    # 合併標籤
    mixup_boxes = np.concatenate([boxes_1, boxes_2], axis=0)
    
    # 將合併的標籤寫入 txt 檔案
    with open(os.path.join(output_folder, f'mixup_{idx}.txt'), 'w') as f:
        for box in mixup_boxes:
            box_str = ' '.join(map(str, box))
            f.write(box_str + '\n')

def main():
    # 設定資料夾路徑和輸出資料夾
    folder_path = r"C:\Users\Ray\Desktop\宜鼎\c4_second_slice"
    output_folder = "Mixup"
    os.makedirs(output_folder, exist_ok=True)

    # 讀取資料夾中的所有圖檔和標籤檔案
    files = os.listdir(folder_path)
    images = [f for f in files if f.endswith('.png')]
    labels = [f for f in files if f.endswith('.txt')]

    # 設置混合數量計數器
    generated_count = 0
    while generated_count < 3000:  # 控制生成混合圖像的數量
        # 隨機選擇一張圖像及其對應的標籤
        image_1_name = random.choice(images)
        image_1_path = os.path.join(folder_path, image_1_name)
        boxes_1 = read_label_file(os.path.join(folder_path, image_1_name.replace('.png', '.txt')))
        
        # 再隨機選擇一張與第一張圖像對應的圖像及其對應的標籤
        image_2_name = random.choice([img for img in images if img != image_1_name])
        image_2_path = os.path.join(folder_path, image_2_name)
        boxes_2 = read_label_file(os.path.join(folder_path, image_2_name.replace('.png', '.txt')))

        mixup(image_1_path, boxes_1, image_2_path, boxes_2, output_folder, generated_count)
        
        generated_count += 1

if __name__ == "__main__":
    main()
